# LangChain and LangSmith Notes

## Import the Supporting Modules for this Notebook

These commands were ran with Python 3.11.8.

To load the dependencies for this project using `pip`, follow these steps:

1. Open a terminal or command prompt.
2. Navigate to the directory where your project is located.
3. Create a virtual environment (optional but recommended):
   - Run `python3 -m venv env` to create a virtual environment named "env".
   - Activate the virtual environment:
     - On Windows, run `env\Scripts\activate`.
     - On macOS and Linux, run `source env/bin/activate`.
   - I use pyenv and the pyenv virtualenv wrapper for this so it is up to you on how you would like to do this. 
4. Install the dependencies:
   - Run `pip install -r requirements.txt` to install the dependencies listed in the "requirements.txt" file.
5. Once the installation is complete, you can use the dependencies in your project.

To load the dependencies for this project using `poetry`, follow these steps:

1. Open a terminal or command prompt.
2. Navigate to the directory where your project is located.
3. If you haven't already, install `poetry` by running `pip install poetry`.
4. Create a virtual environment (optional but recommended):
   - Run `poetry env use python3` to create a virtual environment using Python 3.
   - Activate the virtual environment by running `poetry shell`.
5. Install the dependencies:
   - Run `poetry install` to install the dependencies specified in the "pyproject.toml" file.
   - Poetry will create a virtual environment and install the dependencies within it.
6. Once the installation is complete, you can use the dependencies in your project.


In [ ]:
import os
from random import choice
from uuid import uuid4
from datetime import datetime

from dotenv import load_dotenv
from IPython.display import Markdown, display
from requests import get

## Import LangChain Modules


In [ ]:
from langchain.prompts.chat import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain_anthropic import ChatAnthropic
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_mistralai import ChatMistralAI

### Langchain Globals


In [ ]:
from langchain.globals import set_debug, set_verbose
from langchain.callbacks.tracers import ConsoleCallbackHandler

## Load Keys

Be sure you create the appropiate .env file in your folder. This includes your Langsmith Keys.


In [ ]:
load_dotenv()

anthropic_api_key = os.getenv("ANTHROPIC_API_KEY")
google_api_key = os.getenv("GOOGLE_API_KEY")
mistral_api_key = os.getenv("MISTRAL_API_KEY")
openweather_api_key = os.getenv("OPENWEATHER_API_KEY")
openai_api_key = os.getenv("OPENAI_API_KEY")

## Template Creation


### Base Template


In [ ]:
template = "You are a very friendly and helpful Assistant. Please help the user with their queries. User: {query}"
prompt = ChatPromptTemplate.from_template(template=template)

### Pirate Template


In [ ]:
pirate_template = "Avast, me hearty! Ye be a friendly and helpful Assistant. Translate this sentence into Pirate, the language of the high seas. I be askin' ye to translate this here passage fer me, if ye please. {passage}"
pirate_prompt = ChatPromptTemplate.from_template(template=pirate_template)

## LLM creation

In this example we are using Google Gemini and Anthropics Claude 2.1


In [ ]:
gemini_llm = ChatGoogleGenerativeAI(google_api_key=google_api_key, model="gemini-pro")
anthropic_llm = ChatAnthropic(api_key=anthropic_api_key, model="claude-3-opus-20240229")
mistral_llm = ChatMistralAI(model="mistral-large-latest")

## LCEL Creation

### Basic Query

Here we'll create a few basic invokes and chains for our demonstration.


In [ ]:
# Basic query

basic_query_g = prompt | gemini_llm | StrOutputParser()
basic_query_a = prompt | anthropic_llm | StrOutputParser()
basic_query_m = prompt | mistral_llm | StrOutputParser()

### Chaining

Next let's create a simple chain


In [ ]:
# Simple chain that translates an LLM output into pirate

# User's query -> Google LLM -> Google LLM output -> Anthropic LLM -> Anthropic LLM output in Pirate
anthropic_translates = pirate_prompt | anthropic_llm
pirate_query = {"passage": basic_query_g} | anthropic_translates | StrOutputParser()

### API chain

To make it a little more interesting we can also try an API chain. First let's create a simple function that makes an api call to Openweathermap.org


In [ ]:
# A simple API call to get the current weather


def get_weather(weather_api):
    api_data = weather_api.get("weather_api")
    weather_api_response = get(
        f"https://api.openweathermap.org/data/3.0/onecall?lat={api_data.lat}&lon={api_data.lon}&units={api_data.units}&appid={openweather_api_key}"
    )
    weather_api_response.raise_for_status()
    dt = datetime.fromtimestamp(weather_api_response.json()["current"]["dt"])
    formatted_time = (
        f'{dt.strftime("%I:%M %p")} Timezone={weather_api_response.json()["timezone"]}'
    )
    return {
        "local time": formatted_time,
        "curent weather": weather_api_response.json()["current"],
    }

Above you will notice I'm using the weather_api object in the function above. I will use Pydantic with Langchain. This allows helps ensure that the corret data is given and in a format that is usesable for my function. If I wanted to go further I could have added validation tests to ensure the data is valid for the api.


#### Build the Schema

Here I'd like to take a brief detour here from LCEL and LangSmith and show something that is important as we start working with structured output. LangChain helps us implement a Pydantic Model for our structured output.


In [ ]:
# First create the model for our output parser
from typing import Literal

from langchain_core.pydantic_v1 import BaseModel, Field

Note that the docstrings here are crucial, as they will be passed along to the model along with the class name.


In [ ]:
class WeatherLookup(BaseModel):
    """Multiply two integers together."""

    lat: float = Field(
        ...,
        title="Latitude",
        description="This is the latitude of the location you want to look up the weather for",
    )
    lon: float = Field(
        ...,
        title="Longitude",
        description="This is the longitude of the location you want to look up the weather for",
    )
    units: Literal["imperial", "metric"] = Field(
        "imperial",
        title="Unit",
        description="This is the unit you want the temperature to be in (imperial for F or metric for C)",
    )

We will create a prompt for the user to make a query in natural language to ask for the weather in any desired location. The docsstrings in the Pydantic Model tell the LLM how the output should be structured. While it is possible to just provide the query without instruction, by providing instructions, the LLM can provide more accurate results and more focused responses.


In [ ]:
# This is the template for the LLM to understand our request.
weather_template = "You are a very friendly and helpful Assistant. Please help the user with their weather related queries. User: {query}"
weather_prompt = ChatPromptTemplate.from_template(template=weather_template)

# We will create a weather man prompt for the generative ouptut at the end of the chain

weather_man_template = "This is the response from api.openweather.com about the current weather conditions. Based on this response, can you please create a friendly weatherman report and include relevant emojis that are fun and personalized to me? Also, please provide the current time, which is provided in the API output you have received as well.\n\n{weather_api_response}"
weather_man_prompt = ChatPromptTemplate.from_template(template=weather_man_template)

#### Configure the LLM and the Chain

There are couple things going on here:


In [ ]:
# We will use Mistral for the first step, for the structure of the output
from langchain_core.runnables import RunnableLambda


structured_llm_m = mistral_llm.with_structured_output(WeatherLookup)

retrieve_location_chain = {
    "weather_api": (weather_prompt | structured_llm_m)
} | RunnableLambda(get_weather)

weather_output = weather_man_prompt | gemini_llm | StrOutputParser()

api_chain = {"weather_api_response": retrieve_location_chain} | weather_output

## LangSmith

First let's run the chains and then review them in LangSmith.

To use the out of the box functionality you just need to have the following in your environment:

```ini
# Langchain Settings
LANGCHAIN_TRACING_V2=true
LANGCHAIN_ENDPOINT=https://api.smith.langchain.com
LANGCHAIN_API_KEY=<API_KEY>
LANGCHAIN_PROJECT=barry-local-dev # this is the project name the traces will be found in LangSmith
```


### Run the basic query


In [ ]:
basic_query_g.invoke(
    {
        "query": "Can you create a social media post for my blog about Artificial Intelligence and its impacts on our daily lives?"
    }
)

In [ ]:
basic_query_a.invoke(
    {
        "query": "Can you create a social media post for my blog about Artificial Intelligence and its impacts on our daily lives?"
    }
)

In [ ]:
basic_query_m.invoke(
    {
        "query": "Can you create a social media post for my blog about Artificial Intelligence and its impacts on our daily lives?"
    }
)

### Run the simple chain

Let's run a simple chain.


In [ ]:
# So that we can see the conversation in realtime let's turn on verbose mode
set_verbose(True)

print(
    pirate_query.invoke(
        {
            "query": "In regard to George Lucas's Space opera, Empire Strikes Back, can you recite the dialogue between Darth Vadar and Luke Skywalker during their duel on Bespin when Darth Vadar reveals to Luke that he is Luke's Father? Please provide the dialoge only."
        },
        config={"callbacks": [ConsoleCallbackHandler()]},
    )
)

### Run the API Chain


In [ ]:
print(
    api_chain.invoke(
        {
            "query": "What is the weather like at Disney World today in Orlando, Florida?"
        },
        config={"callbacks": [ConsoleCallbackHandler()]},
    )
)

## Configuring Tagging and Metadata


### Tagging

Let's turn off verbosity for now.


In [ ]:
set_verbose(False)  # Turn off verbose mode

One whay of adding tags and metadata to a chain is the `with_config()` method of the `Runnable` class.


In [ ]:
# Tagging the chains

# We are using the keyword "run_name" to tag the run

basic_query_g = basic_query_g.with_config({"run_name": "Basic Query with Google LLM"})
basic_query_a = basic_query_a.with_config(
    {"run_name": "Basic Query with Anthropic LLM"}
)
basic_query_m = basic_query_m.with_config({"run_name": "Basic Query with Mistral LLM"})

google_output = basic_query_g.invoke(
    {
        "query": "Can you create a social media post for my blog about Artificial Intelligence and its impacts on our daily lives?"
    }
)
anthropic_output = basic_query_a.invoke(
    {
        "query": "Can you create a social media post for my blog about Artificial Intelligence and its impacts on our daily lives?"
    }
)
mistral_output = basic_query_m.invoke(
    {
        "query": "Can you create a social media post for my blog about Artificial Intelligence and its impacts on our daily lives?"
    }
)
display(
    Markdown(
        f"""
                 **Google Output**
                 
                 {google_output}
                 
                 **Anthropic Output**
                 
                 {anthropic_output}
                 
                 **Mistral Output**
                 
                 {mistral_output}
                 """
    )
)

In [ ]:
# Let's tag the remaining chains
priate_query = pirate_query.with_config({"run_name": "Pirate Translate Chain"})
api_chain = api_chain.with_config({"run_name": "Weather API Chain"})

In [ ]:
priate_query.invoke(
    {
        "query": "In regard to George Lucas's Space opera, Empire Strikes Back, can you recite the dialogue between Darth Vadar and Luke Skywalker during their duel on Bespin when Darth Vadar reveals to Luke that he is Luke's Father? Please provide the dialoge only."
    }
)

In [ ]:
api_chain.invoke({"query": "Whats the weather like on Chicago in Farenheit?"})

### Metadata

First the class RunnableConfig will need to be imported.


In [ ]:
from langchain_core.runnables import RunnableConfig

For example, if our application were tracking the user and the session id we can add that as meta data to the conversation. RunnableConfig also can be used to update the the following:

```python
"""Configuration for a Runnable."""

tags: List[str]
"""
Tags for this call and any sub-calls (eg. a Chain calling an LLM).
You can use these to filter calls.
"""

metadata: Dict[str, Any]
"""
Metadata for this call and any sub-calls (eg. a Chain calling an LLM).
Keys should be strings, values should be JSON-serializable.
"""

callbacks: Callbacks
"""
Callbacks for this call and any sub-calls (eg. a Chain calling an LLM).
Tags are passed to all callbacks, metadata is passed to handle*Start callbacks.
"""

run_name: str
"""
Name for the tracer run for this call. Defaults to the name of the class.
"""

max_concurrency: Optional[int]
"""
Maximum number of parallel calls to make. If not provided, defaults to
ThreadPoolExecutor's default.
"""

recursion_limit: int
"""
Maximum number of times a call can recurse. If not provided, defaults to 25.
"""

configurable: Dict[str, Any]
"""
Runtime values for attributes previously made configurable on this Runnable,
or sub-Runnables, through .configurable_fields() or .configurable_alternatives().
Check .output_schema() for a description of the attributes that have been made
configurable.
"""

run_id: Optional[uuid.UUID]
"""
Unique identifier for the tracer run for this call. If not provided, a new UUID
    will be generated.
"""
```


In [ ]:
username = choice(["barweiss@cisco.com", "user1@cisco.com", "user2@cisco.com"])
session_id = str(uuid4())
google_output = basic_query_g.invoke(
    {
        "query": "Can you create a social media post for my blog about Artificial Intelligence and its impacts on our daily lives?"
    },
    RunnableConfig(metadata={"username": username, "session_id": session_id}),
)

username = choice(["barweiss@cisco.com", "user1@cisco.com", "user2@cisco.com"])
session_id = str(uuid4())
anthropic_output = basic_query_a.invoke(
    {
        "query": "Can you create a social media post for my blog about Artificial Intelligence and its impacts on our daily lives?"
    },
    RunnableConfig(metadata={"username": username, "session_id": session_id}),
)

username = choice(["barweiss@cisco.com", "user1@cisco.com", "user2@cisco.com"])
session_id = str(uuid4())
mistral_output = basic_query_m.invoke(
    {
        "query": "Can you create a social media post for my blog about Artificial Intelligence and its impacts on our daily lives?"
    },
    RunnableConfig(metadata={"username": username, "session_id": session_id}),
)
display(
    Markdown(
        f"""
                 **Google Output**
                 
                 {google_output}
                 
                 **Anthropic Output**
                 
                 {anthropic_output}
                 
                 **Mistral Output**
                 
                 {mistral_output}
                 """
    )
)

Tagging Prompts, Output Parsers, etc
More than just LLM and Run can be tagged.


In [ ]:
username = choice(["barweiss@cisco.com", "user1@cisco.com", "user2@cisco.com"])
session_id = str(uuid4())

template = "You are a very friendly and helpful Assistant. Please help the user with their queries. User: {query}"
prompt = ChatPromptTemplate.from_template(template=template)
tagging_test = (
    prompt.with_config({"tags": ["friendly", "helpful"]})
    | mistral_llm.with_config({"run_name": "Friendly_Mistral"})
    | StrOutputParser()
)
tagging_test = tagging_test.with_config({"run_name": "Friendly_Mistral"})
tagging_test.invoke(
    {
        "query": "Can you create a social media post for my blog about Artificial Intelligence and its impacts on our daily lives?"
    },
    config=RunnableConfig(metadata={"username": username, "session_id": session_id}),
)

## LangSmith SDK and Tracer


In [ ]:
from langchain_core.tracers.context import tracing_v2_enabled
from langsmith import Client

Client for interacting with the LangSmith API.


In [ ]:
client = Client()
print(client.info, "\n")
print(client.read_project(project_name="barry-local-dev"), "\n")

Retrieve the tests results of a all runs a project as a panadas DataFrame object. Pandas needs to be installed.


In [ ]:
df = client.get_test_results(project_name="barry-local-dev")
df.head()

With the tracer used in a context mannager you can leverage a call back to get information back about a run without logging to LangSmith.


In [ ]:
with tracing_v2_enabled() as cb:
    prompt = ChatPromptTemplate.from_template(
        template="You are a helpful and harmless AI assistant. \n\nHuman: {query}"
    )
    basic_query_a = (
        prompt.with_config({"tags": ["Tracing Example"]})
        | anthropic_llm.with_config(
            {"run_name": "Anthropic-Example", "tags": ["Tracing Example"]}
        )
        | StrOutputParser().with_config({"tags": ["Tracing Example"]})
    )
    basic_query_a = basic_query_a.with_config({"run_name": "Tracing Example"})

    output = basic_query_a.invoke(
        {
            "query": "Can you create a social media post for my blog about Artificial Intelligence and its impacts on our daily lives?"
        }
    )
    print(f"Output: {output}\n\n")
    print(f"UUID FOR LATEST ID: {cb.latest_run.id}")
    print(f"URL LINK TO RUN {cb.latest_run.id}: {cb.get_run_url()}")

Using the SDK to get the run information as a `Run` object.


In [ ]:
output = client.read_run(cb.latest_run.id)
output

### Evaluators


In [ ]:
import langsmith
from langchain import smith
from langchain_openai import ChatOpenAI

# Replace with the chat model you want to test
my_llm = ChatGoogleGenerativeAI(google_api_key=google_api_key, model="gemini-pro")

# Define the evaluators to apply
eval_config = smith.RunEvalConfig(
    evaluators=["cot_qa"],
    custom_evaluators=[],
    eval_llm=ChatOpenAI(
        model="gpt-4-turbo", temperature=0, openai_api_key=openai_api_key
    ),
)

client = langsmith.Client()
chain_results = client.run_on_dataset(
    dataset_name="Weather API Generative Output",
    llm_or_chain_factory=my_llm,
    evaluation=eval_config,
    project_name=str(uuid4()),
    concurrency_level=5,
    verbose=True,
)